In [7]:
import pandas as pd

In [8]:
df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\wine-clustering.csv',encoding='unicode_escape')
df

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

In [10]:
num_clusters = 3
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

In [11]:
def run_pygad(crossover_type, crossover_prob, mutation_prob, sol_per_pop, num_generations, num_parents_mating):
    ga_instance = pygad.GA(
        num_generations=num_generations,
        num_parents_mating=num_parents_mating,
        crossover_probability=crossover_prob,
        mutation_probability=mutation_prob,
        sol_per_pop=sol_per_pop,
        num_genes=num_genes,
        fitness_func=fitness_func,
        parent_selection_type="rws",  # Roulette Wheel Selection
        crossover_type=crossover_type
    )
    # Run the genetic algorithm
    ga_instance.run()

    # Get the best solution and its fitness value
    best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()

    return best_solution, best_solution_fitness, best_solution_idx

In [12]:
grid_search_results = []
crossover_types = ['single_point', 'two_points', 'uniform']
crossover_probabilities = [0.6, 0.7, 0.8, 0.9]
mutation_probabilities = [0.01, 0.05, 0.1, 0.2]
population_sizes = [10, 20, 30, 40]
num_parents_mating_values = [2, 3, 4]

for crossover_type in crossover_types:
    for crossover_prob in crossover_probabilities:
        for mutation_prob in mutation_probabilities:
            for sol_per_pop in population_sizes:
                for num_parents_mating in num_parents_mating_values:
                    num_generations = 1000
                    sol,fitness,index = run_pygad(crossover_type,crossover_prob, mutation_prob, sol_per_pop, num_generations,
                                                num_parents_mating)
                    grid_search_results.append({
                        'crossover_type':crossover_type,
                        'crossover_probability':crossover_prob,
                        'mutation_probability':mutation_prob,
                        'sol_per_pop':sol_per_pop,
                        'num_parents_mating':num_parents_mating,
                        'sol':sol,
                        'fitness':fitness,
                    })
            
grid_search_results.sort(key=lambda x: x['fitness'])
best_hyperparameters = grid_search_results[0]
print(best_hyperparameters)


{'crossover_type': 'single_point', 'crossover_probability': 0.8, 'mutation_probability': 0.01, 'sol_per_pop': 10, 'num_parents_mating': 2, 'sol': array([ 7.70177824e-01,  3.93863042e-03, -2.96410370e+00, -5.99036127e+00,
       -2.25962318e+00, -1.44822079e+00, -5.36306168e-02,  7.15930653e-01,
       -3.56016948e+00,  3.01463482e+00, -2.77113013e+00, -2.29786536e+00,
       -1.72768429e+00,  1.39489247e+00,  5.15445942e+00,  6.75865249e-01,
       -1.07241776e-01,  2.39424323e-01, -4.51282480e+00,  2.51444280e+00,
       -2.17721709e+00, -1.60158338e+00, -1.41770402e+00, -4.26944193e+00,
       -1.55897131e+00,  4.82085131e+00,  1.68746822e+00, -1.67370568e+00,
       -1.61604928e+00,  1.41533419e+00,  1.27661796e+01, -1.20525264e+00,
        5.95047691e-01,  4.27841933e-01,  1.54957334e+00, -6.72607490e-01,
        1.38566390e-01, -2.30253176e+00,  1.48405398e+01]), 'fitness': 7.607775690490711e-06}


In [13]:
_, _, cluster_indices, _, _ = cluster_data(best_hyperparameters['sol'])
print(cluster_indices)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
